In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
main_url = 'https://www.efsyn.gr'

start_url = 'https://www.efsyn.gr/syriza-proodeytiki-symmahia?page='

start_page = 1
end_page = 6

In [ ]:
response = requests.get(start_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:

teasers_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  teaser_articles_list = doc.find('div', {'class': 'default-teaser triple'}).find_all('article')


  for teaser in teaser_articles_list:
      story_dict = {}
      try:

        link = teaser.find('a', {'class': 'full-link'})['href']


        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)
  time.sleep(1)


In [ ]:
efsyn_teasers_df = pd.DataFrame(teasers_list)
efsyn_teasers_df

,url
0,https://www.efsyn.gr/politiki/antipoliteysi/39...
1,https://www.efsyn.gr/politiki/antipoliteysi/39...
2,https://www.efsyn.gr/politiki/antipoliteysi/39...
3,https://www.efsyn.gr/politiki/antipoliteysi/39...
4,https://www.efsyn.gr/politiki/kybernisi/396841...
...,...
67,https://www.efsyn.gr/politiki/antipoliteysi/39...
68,https://www.efsyn.gr/politiki/antipoliteysi/39...
69,https://www.efsyn.gr/politiki/antipoliteysi/39...
70,https://www.efsyn.gr/politiki/antipoliteysi/39...


In [ ]:
article_url = 'https://www.efsyn.gr/politiki/antipoliteysi/395643_syriza-apofasistikotita-kontra-ston-kindyno-esostrefeias'

In [ ]:
response = requests.get(article_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:
full_articles_list = []

for article_url in efsyn_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  article = doc.find('section', {'class': 'article__main'})

  full_article_dict = {}

  full_article_dict['site'] = "efsyn.gr"

  full_article_dict['url'] = article_url

  try:
    section = doc.find('a', {'class': 'article__category'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  try:
    title = title = article.find('h1').text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  try:
    date = article.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  try:
    author = article.find('div', {'class', "article__author"}).span.text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)


efsyn_full_articles_df = pd.DataFrame(full_articles_list)
efsyn_full_articles_df

,site,url,section,title,date,author,full_text
0,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,Εν μέσω έντονων «ζυμώσεων» συνεδριάζει η Πολιτ...,2023-07-09 20:19,efsyn.gr,Κλιμακώνονται οι διεργασίες για την «επόμενη μ...
1,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,ΣΥΡΙΖΑ: Πρόεδρος πριν ή μετά τις αυτοδιοικητικές;,2023-07-09 13:09,efsyn.gr,Σε πλήρη εξέλιξη οι διεργασίες για την επόμενη...
2,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,ΣΥΡΙΖΑ: Τάσεις με... αποστάσεις,2023-07-09 09:48,Δημήτρης Τερζής,Μία εβδομάδα απέμεινε ώς την κρίσιμη συνεδρίασ...
3,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,Σωκράτης Φάμελλος: «Ψήφος εμπιστοσύνης στην κο...,2023-07-08 19:44,efsyn.gr,Σε πλήρη «διύλιση» των προγραμματικών δηλώσεων...
4,efsyn.gr,https://www.efsyn.gr/politiki/kybernisi/396841...,ΚΥΒΕΡΝΗΣΗ,"Οι γάμοι ομοφυλοφίλων, ο πρωθυπουργός, ο Μ. Βο...",2023-07-08 12:49,Α.Κ.,Σφοδρή αντίδραση από τον ΣΥΡΙΖΑ-Προοδευτική Συ...
...,...,...,...,...,...,...,...
67,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,Live οι έκτακτες δηλώσεις Τσίπρα - Πληροφορίες...,2023-06-29 11:22,efsyn.gr,Σ' εξέλιξη βρίσκονται οι δηλώσεις του Αλέξη Τσ...
68,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,Ο Αλέξης Τσίπρας ανοίγει τα χαρτιά του,2023-06-28 22:53,efsyn,Αλέξης Τσίπρας Έπειτα από τρεις μέρες εσωτερικ...
69,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,ΣΥΡΙΖΑ: Αποφασιστικότητα κόντρα στον κίνδυνο ε...,2023-06-28 20:21,efsyn.gr,Εν αναμονή των συνεδριάσεων των οργάνων του ΣΥ...
70,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,Παρέμβαση Τεμπονέρα με αποδέκτη και την Κεντρο...,2023-06-28 16:09,efsyn.gr,Μήνυμα προς το ΠΑΣΟΚ-ΚΙΝ.ΑΛΛ. στέλνει ο συντον...


In [ ]:
efsyn_full_articles_df['datetime'] = pd.to_datetime(efsyn_full_articles_df['date'], format='%Y-%m-%d %H:%M')
efsyn_full_articles_df.head()

,site,url,section,title,date,author,full_text,datetime
0,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,Εν μέσω έντονων «ζυμώσεων» συνεδριάζει η Πολιτ...,2023-07-09 20:19,efsyn.gr,Κλιμακώνονται οι διεργασίες για την «επόμενη μ...,2023-07-09 20:19:00
1,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,ΣΥΡΙΖΑ: Πρόεδρος πριν ή μετά τις αυτοδιοικητικές;,2023-07-09 13:09,efsyn.gr,Σε πλήρη εξέλιξη οι διεργασίες για την επόμενη...,2023-07-09 13:09:00
2,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,ΣΥΡΙΖΑ: Τάσεις με... αποστάσεις,2023-07-09 09:48,Δημήτρης Τερζής,Μία εβδομάδα απέμεινε ώς την κρίσιμη συνεδρίασ...,2023-07-09 09:48:00
3,efsyn.gr,https://www.efsyn.gr/politiki/antipoliteysi/39...,ΑΝΤΙΠΟΛΙΤΕΥΣΗ,Σωκράτης Φάμελλος: «Ψήφος εμπιστοσύνης στην κο...,2023-07-08 19:44,efsyn.gr,Σε πλήρη «διύλιση» των προγραμματικών δηλώσεων...,2023-07-08 19:44:00
4,efsyn.gr,https://www.efsyn.gr/politiki/kybernisi/396841...,ΚΥΒΕΡΝΗΣΗ,"Οι γάμοι ομοφυλοφίλων, ο πρωθυπουργός, ο Μ. Βο...",2023-07-08 12:49,Α.Κ.,Σφοδρή αντίδραση από τον ΣΥΡΙΖΑ-Προοδευτική Συ...,2023-07-08 12:49:00


In [ ]:
efsyn_full_articles_df.to_csv('/content/gdrive/MyDrive/DATA_JOUR/FINAL ESSAY/efsyn_full.csv', index=False)


NameError: ignored